In [1]:
import torch
import json
import pandas as pd

from scipy.signal import decimate
from math import ceil, floor, log, exp
import json
from scipy.stats import binom
from tqdm import tqdm

# Extracting all Seizure Info

In [2]:
# Hack to get patient IDs
patient_ids = list(torch.load("data/interictal.pt").keys())

In [12]:
seizure_info = pd.DataFrame(columns=["patient_id", "start_time", "end_time", "note"])
for patient_id in patient_ids:
    # Load the annotation for the patients recording
    labels = pd.read_csv(f"data\data\{patient_id}\{patient_id}_labels.csv")

    # Remove the first and last hour of recording
    first_hour_indexes = labels["labels.startTime"] - labels["startTime"]<= 3600000
    last_hour_indexes = labels["labels.startTime"] - labels["startTime"] >= labels["duration"] - 3600000
    labels = labels[~first_hour_indexes&~last_hour_indexes]

    # Remove subclinical events from labels
    with open("labels.json") as f:
        label_notes = json.load(f)

    seizure_indexes = [label_notes[note]==1 for note in labels["labels.note"].values]
    labels = labels[seizure_indexes]

    # Add patient's seizures to seizure data
    previous_seizure_time = 0
    for i in labels.index:
        start_time = labels.loc[i, "labels.startTime"] - labels.loc[i, "startTime"]
        end_time = start_time + labels.loc[i, "labels.duration"]
        note = labels.loc[i, "labels.note"]
        is_lead_seizure = (start_time > previous_seizure_time + 7200000)
        seizure_info = seizure_info.append({"patient_id": patient_id, "start_time": start_time, "end_time": end_time, "note": note, "is_lead_seizure": is_lead_seizure}, ignore_index=True)
        previous_seizure_time = end_time
seizure_info = seizure_info.drop_duplicates()
seizure_info.to_csv("data/seizures.csv")

# Writing Test Segments for each patient

In [3]:
for test_patient in patient_ids:
    test_data = pd.read_parquet(f"data\{test_patient}_128Hz_scaled.parquet")
    test_data = test_data.iloc[460800:-460800]
    
    patient_segments = []
    segment_times = []
    max_time = test_data.index[-1]
    segment_start = 3600000.0
    segment_end = segment_start + 29992.1875
    while segment_end < max_time:
        segment_data = test_data[segment_start:segment_end]
        for channel in segment_data.columns[segment_data.isna().any(axis=0)]:
            if segment_data[channel].isna().all():
                segment_data[channel] = 0.5
            else:
                segment_data[channel].fillna(segment_data[channel].mean(), inplace=True)
        
        downsampled = decimate(segment_data, q=8, axis=0)
        downsampled = decimate(downsampled, q=4, axis=0)

        patient_segments.append(downsampled)
        segment_times.append(segment_start+30000)

        segment_start += 30000
        segment_end = segment_start + 29992.1875
    segments_tensor = torch.Tensor(patient_segments)

    torch.save(segments_tensor, f"data/test/{test_patient}_test_segments.pt")

  0%|          | 0/23 [00:00<?, ?it/s]C:\Users\Bailey\anaconda3\envs\torch\lib\site-packages\pandas\core\series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
<ipython-input-3-91f3c2b663fc>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  segment_data[channel] = 0.5
100%|██████████| 23/23 [08:13<00:00, 21.45s/it]
